In [1]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from common import create_engine, display_all

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters() # converters e.g. for datetime in plots

In [2]:
engine = create_engine('db-conf.json', 'dp.kamko.dev')
pd.sql = lambda sql: pd.read_sql(sql, con=engine)

Najnovsie info o facebook engagemente pre sledovane URL

In [3]:
df = pd.sql('''
SELECT sq.*, s.url as source
FROM (
         SELECT afb.*, row_number() over (partition by url order by sync_date desc) rn
         FROM article_fb_engagement afb) sq
JOIN article a on sq.url = a.url
JOIN source s on a.source_id = s.id
WHERE sq.rn = 1;
''')
df['popularity'] = df['reaction_count'] + df['comment_count'] + df['share_count'] + df['comment_plugin_count']

In [4]:
display_all(df.groupby('source')['popularity'].describe().sort_values('count', ascending=False))

,count,mean,std,min,25%,50%,75%,max
source,,,,,,,,
http://naturalnews.com,20945.0,1225.714825,41313.845801,0.0,19.00,88.0,250.00,5197586.0
http://naturalnewsblogs.com,17756.0,86.888263,2294.673255,0.0,0.00,0.0,1.00,203855.0
http://neurosciencenews.com,13148.0,271.558564,4006.532868,0.0,0.00,0.0,45.25,294411.0
http://statnews.com,12275.0,211.472505,1868.624163,0.0,0.00,0.0,23.00,82773.0
http://hsionline.com,8175.0,0.821162,12.145379,0.0,0.00,0.0,0.00,726.0
http://naturalsociety.com,7350.0,10.235374,598.635593,0.0,0.00,0.0,0.00,49319.0
http://healthnutnews.com,7249.0,1657.439095,23514.407619,0.0,4.00,179.0,741.00,1810343.0
http://healthimpactnews.com,6267.0,354.368917,10890.051283,0.0,0.00,0.0,9.00,841807.0
http://infowars.com,6024.0,52.932105,380.116241,0.0,0.00,0.0,8.00,16902.0


In [5]:
def show_source_popularity_histogram(df, source, quantile):
    xdf = df[df['source'] == source]
    xdf = xdf[xdf['popularity'] < xdf['popularity'].quantile(quantile)]
    plot = sns.distplot(xdf['popularity'])
    plt.show()
    return plot

sources_sorted = df.groupby('source')['popularity'].describe().sort_values('count', ascending=False).index

In [6]:
rsc = df[['url', 'reaction_count', 'share_count', 'comment_count']]

In [7]:
rsc.corr()

,reaction_count,share_count,comment_count
reaction_count,1.000000,0.883691,0.762594
share_count,0.883691,1.000000,0.861779
comment_count,0.762594,0.861779,1.000000


Normalizovana popularita voci casu (x = 0 == datum najskorsieho clanku)

In [8]:
monitored_articles = pd.sql('''
SELECT afe.*,
       COALESCE(art.published_at, (SELECT min(sync_date) FROM article_fb_engagement safe WHERE art.url = safe.url)) published_at,
       row_number() over (PARTITION BY afe.url ORDER BY sync_date)
FROM article_fb_engagement afe
         JOIN article art on afe.url = art.url
WHERE afe.url in (SELECT url
                  FROM article
                           JOIN monitored_article ma on article.id = ma.article_id)
ORDER BY afe.url, sync_date;
''')

monitored_articles.sync_date = monitored_articles.sync_date + pd.DateOffset(hours=2) # sync_date is in UTC and published_at is in UTC+2 (I hope)
monitored_articles = monitored_articles.drop(columns=['comment_plugin_count', 'row_number', 'id'])
monitored_articles['popularity'] = monitored_articles['reaction_count'] + monitored_articles['comment_count'] + monitored_articles['share_count'] 

In [9]:
# add artifical first row for every value where engagement = (0,0,0) and sync_date = published_at
_mart = monitored_articles[['url','published_at']]
_mart = _mart.drop_duplicates()

for row in _mart.itertuples():
    new_row = [row.url, 0, 0, 0, row.published_at, row.published_at]
    monitored_articles.append(new_row)

Podmnozina clankov pre jasnejsi diagram

In [ ]:
sample_size = 50

urls = pd.Series(monitored_articles.url.unique()).sample(n=sample_size, random_state=1233)
sampled_articles = monitored_articles[monitored_articles.url.isin(urls)]

In [ ]:
min_time = sampled_articles['sync_date'].min()
rplot = sns.relplot(data=sampled_articles,
            x='sync_date', y='popularity',
            hue='url', kind='line',
            height=12, aspect=1,
            legend=False)
rplot.set(xlim=(min_time, None))
rplot